# Proyek Prediksi Banjir Samarinda

**Lingkup Proyek:**
1.  **Lokasi:** Kota Samarinda.
2.  **Output:** Klasifikasi Biner (1 = Potensi Banjir, 0 = Aman).
3.  **Data:** Data cuaca historis (Open-Meteo) & Data kejadian banjir (Labeling manual).

Notebook ini mencakup:
1.  Pengumpulan Data (Data Collection)
2.  Pembersihan & Persiapan Data (Data Cleaning)
3.  Rekayasa Fitur (Feature Engineering)
4.  Pelatihan Model (Model Training)
5.  Evaluasi Model
6.  Penyimpanan Model (Export)

In [ ]:
import requests
import pandas as pd
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date, timedelta
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

# Konfigurasi Plot
sns.set(style="whitegrid")

## 1. Data Collection (Pengumpulan Data)
Mengambil data cuaca historis dari Open-Meteo API.

In [ ]:
def get_historical_weather_samarinda(years=5):
    print("⏳ Sedang mengambil data historis dari Open-Meteo...")

    # 1. Setup Lokasi Samarinda & Rentang Waktu
    lat = -0.5022
    lon = 117.1536

    end_date = date.today()
    start_date = end_date - timedelta(days=365 * years)

    # URL API Open-Meteo (Archive)
    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        # Variabel Cuaca Penting untuk Banjir:
        "daily": ["precipitation_sum", "rain_sum", "precipitation_hours"],
        "timezone": "Asia/Singapore" # WITA
    }

    # 2. Request Data
    try:
        response = requests.get(url, params=params)
        data = response.json()

        # 3. Parsing ke DataFrame
        daily_data = data['daily']

        df = pd.DataFrame({
            "tanggal": daily_data['time'],
            "curah_hujan_mm": daily_data['precipitation_sum'],
            "durasi_hujan_jam": daily_data['precipitation_hours']
        })

        # Konversi kolom tanggal ke tipe datetime
        df['tanggal'] = pd.to_datetime(df['tanggal'])

        return df

    except Exception as e:
        print(f"Error: {e}")
        return None

# --- Eksekusi ---
df_cuaca = get_historical_weather_samarinda(years=5)

if df_cuaca is not None:
    print(f"\n✅ Berhasil menarik {len(df_cuaca)} baris data!")
    print(df_cuaca.head())
    
    # Simpan ke CSV sebagai backup
    df_cuaca.to_csv("dataset_banjir_samarinda_5tahun.csv", index=False)
    print("\n📁 Data disimpan ke 'dataset_banjir_samarinda_5tahun.csv'")
else:
    print("Gagal mengambil data.")

## 2. Data Loading & Cleaning
Memuat data yang sudah diberi label (jika ada) dan menggabungkannya dengan data cuaca.

In [ ]:
print(f"Current Working Directory: {os.getcwd()}")

# --- Helper Function untuk Load Data ---
def load_data_safe(filename):
    # 1. Coba load dari current directory
    if os.path.exists(filename):
        return pd.read_csv(filename)
    
    # 2. Coba load dari absolute path project (Fallback)
    # Path ini spesifik untuk environment user saat ini
    project_path = "/Users/bidang4/dashboard_eksekutif/prediksi_banjir/"
    abs_path = os.path.join(project_path, filename)
    if os.path.exists(abs_path):
        print(f"ℹ️ File ditemukan di absolute path: {abs_path}")
        return pd.read_csv(abs_path)
    
    return None

# --- Load Data Labeling ---
df_label = load_data_safe('LEMBAR_KERJA_BANJIR.csv')

if df_label is not None:
    print("✅ File labeling ditemukan.")
else:
    print("⚠️ File labeling tidak ditemukan, menggunakan data cuaca mentah saja.")
    df_label = pd.DataFrame(columns=['tanggal', 'is_banjir'])

# --- Load Data Cuaca ---
df_cuaca = load_data_safe('dataset_banjir_samarinda_5tahun.csv')

if df_cuaca is not None:
    print("✅ Data cuaca berhasil dimuat.")
else:
    print("⚠️ File data cuaca tidak ditemukan. Pastikan Anda sudah menjalankan 'Langkah 1: Data Collection' di atas.")

# Cek apakah df_cuaca berhasil dimuat
if df_cuaca is None:
    raise FileNotFoundError("❌ Data cuaca tidak tersedia. Harap jalankan cell 'Data Collection' di atas terlebih dahulu!")

# Gabungkan (Concat) atau Merge jika perlu
df = df_cuaca.copy()
df['tanggal'] = pd.to_datetime(df['tanggal'])

# Merge dengan label jika ada
if not df_label.empty:
    df_label['tanggal'] = pd.to_datetime(df_label['tanggal'])
    # Update is_banjir di df berdasarkan df_label
    df = pd.merge(df, df_label[['tanggal', 'is_banjir']], on='tanggal', how='left')
    # Isi NaN is_banjir dengan 0 (Asumsi tidak ada info = Aman)
    df['is_banjir'] = df['is_banjir'].fillna(0).astype(int)
else:
    # Jika tidak ada label, kita buat dummy logic untuk demo (HAPUS BAGIAN INI JIKA SUDAH ADA DATA REAL)
    print("⚠️ Membuat label dummy berdasarkan curah hujan > 50mm (HANYA UNTUK DEMO)")
    df['is_banjir'] = (df['curah_hujan_mm'] > 50).astype(int)

print(f"Total Data: {len(df)}")
print(df['is_banjir'].value_counts())

## 3. Feature Engineering
Membuat fitur-fitur baru yang relevan untuk prediksi banjir:
- `hujan_kemarin`: Curah hujan H-1
- `hujan_akumulasi_3hari`: Total hujan H-0, H-1, H-2

In [ ]:
# Sort berdasarkan tanggal
df = df.sort_values('tanggal')

# Lag-1: Hujan Kemarin
df['hujan_kemarin'] = df['curah_hujan_mm'].shift(1)

# Rolling Sum 3 Days: Akumulasi hujan 3 hari terakhir (Hari ini + Kemarin + Lusa Kemarin)
df['hujan_akumulasi_3hari'] = df['curah_hujan_mm'].rolling(window=3).sum()

# Drop NaN akibat shifting (baris awal)
df_model = df.dropna().copy()

print(f"Data siap training: {len(df_model)} baris")
df_model.head()

## 4. Model Training
Menggunakan Logistic Regression.

In [ ]:
# Fitur yang digunakan
features = ['curah_hujan_mm', 'durasi_hujan_jam', 'hujan_kemarin', 'hujan_akumulasi_3hari']
target = 'is_banjir'

X = df_model[features]
y = df_model[target]

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Inisialisasi & Training Model
model = LogisticRegression(class_weight='balanced', solver='liblinear')
model.fit(X_train, y_train)

print("✅ Model berhasil dilatih.")

## 5. Evaluasi Model

In [ ]:
y_pred = model.predict(X_test)

print("=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=['Aman', 'Banjir']))

print("\n=== Koefisien Fitur ===")
coef_df = pd.DataFrame({'Fitur': features, 'Koefisien': model.coef_[0]})
print(coef_df.sort_values(by='Koefisien', ascending=False))

In [ ]:
# Optimasi Threshold menggunakan F1-Score
y_scores = model.predict_proba(X_test)[:, 1]
precisions, recalls, thresholds = precision_recall_curve(y_test, y_scores)

f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
f1_scores = np.nan_to_num(f1_scores)

best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]

print(f"Threshold Optimal: {best_threshold:.4f}")

## 6. Simpan Model (Export)
Menyimpan model, threshold, dan nama fitur agar bisa digunakan di aplikasi backend (`main.py`).

In [ ]:
model_pack = {
    "model": model,
    "threshold": best_threshold,
    "features": features
}

joblib.dump(model_pack, "banjir_model_v1.pkl")
print("✅ Model disimpan ke 'banjir_model_v1.pkl'")